[1] 모듈 로딩 및 데이터 준비

In [50]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

[2] 사용자 정의 데이터셋 생성

In [51]:
# 데이터 준비
filename = '../DATA/iris.csv'

irisDF = pd.read_csv(filename)
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [52]:
irisNP = np.loadtxt(filename, delimiter = ',', usecols = [0, 1, 2, 3], skiprows = 1)
irisNP[:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

[2-1] 사용자 정의 데이터셋 클래스

In [53]:
# 데이터의 Tensor 변환
class DLDataset(Dataset):

    # 초기화 콜백 함수(callback function)
    def __init__(self, x_data, y_data):
        super().__init__()
        # x, y 데이터 => ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data

        # ndarray => tensor
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)  # 타겟을 원핫인코딩하면 FloatTensor 사용
        print('[target & target shape]', self.target.shape, self.target.ndim)
    
    # 데이터셋의 개수 체크 콜백 함수(callback function)
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터 + 라벨 반환 콜백 함수(callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

[2-2] 데이터셋 인스턴스 생성

In [54]:
# 피처와 라벨 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF[irisDF.columns[-1]]

print(f'featureDF => {featureDF.shape}, {featureDF.ndim}D')
print(f'targerSR => {targetSR.shape}, {targetSR.ndim}D')

featureDF => (150, 4), 2D
targerSR => (150,), 1D


In [55]:
# 타겟이 object이므로 정수로 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetSR)
targetNP = targetNP.reshape(-1, 1)
print(targetNP.shape, targetNP.ndim)

(150, 1) 2


In [56]:
# 데이터셋 생성 => DF, NP
my_dataset = DLDataset(featureDF, targetNP)
my_dataset[0]

[target & target shape] torch.Size([150, 1]) 2


(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0]))

In [57]:
# 데이터셋 생성 => NP, NP
my_dataset2 = DLDataset(irisNP, targetNP)
my_dataset2[0]

[target & target shape] torch.Size([150, 1]) 2


(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0]))

[2-3] 학습용, 검증용, 테스트용 Dataset

In [58]:
# 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
seed = torch.Generator().manual_seed(42)

trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator = seed)

print(f'trainDS =>{len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개')

print(f'Subset 속성 =>\nindices : {trainDS.indices} \nindices : {trainDS.dataset}')
print(f'Subset 속성 =>\nindices : {validDS.indices} \nindices : {validDS.dataset}')
print(f'Subset 속성 =>\nindices : {testDS.indices} \nindices : {testDS.dataset}')

trainDS =>105개, validDS => 15개, testDS => 30개
Subset 속성 =>
indices : [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12] 
indices : <__main__.DLDataset object at 0x000002029B6F9940>
Subset 속성 =>
indices : [22, 104, 81, 1, 103, 125, 85, 2, 96, 128, 27, 118, 77, 110, 146] 
indices : <__main__.DLDataset object at 0x000002029B6F9940>
Subset 속성 =>
indices : [72, 139, 131, 60, 65, 92, 135, 83, 14, 34, 137, 10, 119, 9, 148, 79, 78, 70, 144, 143, 123, 115, 61, 132, 90, 46, 126, 136, 33, 138] 
indices : <__main__.DLDataset object at 0x000002029B6F9940>


[3] DataLoader 생성 : 학습용, 검증용, 테스트용

In [59]:
# DataLoader 생성
# drop_last 매개변수 : 배치 사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False]
batch = 10
trainDL = DataLoader(trainDS, batch_size = batch, drop_last = True)
validDL = DataLoader(validDS, batch_size = batch)
testDL = DataLoader(testDS, batch_size = batch)

len(trainDL), len(validDL), len(testDL)

(10, 2, 3)

In [60]:
# Epoch당 반복 단위
print(f'batch_size = {batch}')
print(f'trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개')
print(f'trainDL => {len(trainDL)}개, validDL => {len(validDL)}개, testDL => {len(testDL)}개')

batch_size = 10
trainDS => 105개, validDS => 15개, testDS => 30개
trainDL => 10개, validDL => 2개, testDL => 3개


In [62]:
# DataLoader 속성
for _, (feature, target) in enumerate(validDL):
    print(f'[{_}] feature {feature}')

[0] feature tensor([[4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.8000, 2.2000],
        [5.5000, 2.4000, 3.7000, 1.0000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [6.3000, 2.9000, 5.6000, 1.8000],
        [7.2000, 3.2000, 6.0000, 1.8000],
        [6.0000, 3.4000, 4.5000, 1.6000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [5.7000, 2.9000, 4.2000, 1.3000],
        [6.4000, 2.8000, 5.6000, 2.1000]])
[1] feature tensor([[5.2000, 3.5000, 1.5000, 0.2000],
        [7.7000, 2.6000, 6.9000, 2.3000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [6.5000, 3.2000, 5.1000, 2.0000],
        [6.3000, 2.5000, 5.0000, 1.9000]])


[4] Model 클래스 정의 : 입출력 피처 수, 층 수, 은닉층의 노드 수
- 구조 설계
    * 입력층 : 입력 => 피처 개수, iris 데이터의 경우 4개
    * 은닉층 : 사용자가 직접 조정
    * 출력층 : [분류] => 타겟 클래스 개수, [회귀] => 1개

In [63]:
# 모델 클래스 정의
# 클래스명 : CModel
class CModel(nn.Module):

    # 구성요소 정의 함수
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.input_layer = nn.Linear(in_dim, 100)
        self.relu = nn.ReLU()
        self.hidden_layer = nn.Linear(100, 27)
        self.output_layer = nn.Linear(27, out_dim)
    
    # 순방향 학습 진행 함수
    def forward(self, x):
        y = self.input_layer(x) 
        y = self.relu(y)  # 결과 100개 반환
        y = self.hidden_layer(y)
        y = self.relu(y) 
        y = self.output_layer(y)
        return y

[5] 학습 준비 : 실행디바이스, 모델, 최적화, 손실함수, 학습횟수, 학습함수, 평가함수, 예측함수

In [64]:
# 실행 디바이스 선택
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 50

In [65]:
my_dataset2.target.shape

torch.Size([150, 1])

In [66]:
# 모델 인스턴스
IN_DIM, OUT_DIM = my_dataset2.feature.shape[1], targetSR.nunique()
model = CModel(IN_DIM, OUT_DIM)
print(f'IN : {IN_DIM}, OUT : {OUT_DIM}')

IN : 4, OUT : 3


In [67]:
print(model)

CModel(
  (input_layer): Linear(in_features=4, out_features=100, bias=True)
  (relu): ReLU()
  (hidden_layer): Linear(in_features=100, out_features=27, bias=True)
  (output_layer): Linear(in_features=27, out_features=3, bias=True)
)


In [68]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
# 손실함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())
SCHEDULER = ReduceLROnPlateau(OPTIMIZER, mode = 'min', patience = 3)

- 학습 및 검증관련 함수 정의

In [69]:
import torchmetrics.functional as metrics

In [70]:
# 학습 진행함수
def training():
    # 학습모드 => 정규화, 경사하강법, 드롭아웃 등의 기능 활성화
    model.train()
    # 배치크기만큼 학습 진행 및 저장
    train_loss = []
    for cnt, (feature, target) in enumerate(trainDL):
        # print(cnt, feature, target)
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        target = target.squeeze()  # 원핫으로 만들면 필요없음

        # 학습
        pre_target = model(feature)
        # print(f'pre_target => {pre_target.shape}, {pre_target.ndim}D')
        # print(f'target => {target.shape}, {target.ndim}D')

        # 손실함수
        loss = LOSS_FN(pre_target, target)
        train_loss.append(loss)
        
        # W, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

        # 배치 단위 학습 진행 메세지 출력
        # print(f'[Train {cnt} Batch Loss] ==> {loss}')

    # 에포크 단위 학습 진행 메세지 출력
    acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes = 3)
    print(f'[Train Loss] ==> {loss} ')

    return train_loss

In [71]:
# 검증 및 평가 진행함수
# 매개변수 dataLoader : 검증 또는 테스트 데이터셋에 대한 Loader
def testing(dataLoader):
    # 추론모드 => 정규화, 경사하강법, 드롭아웃 등의 기능 활성화
    model.eval()
    
    with torch.no_grad():
        # 배치크기만큼 학습 진행 및 저장
        val_loss = []
        for cnt, (feature, target) in enumerate(dataLoader):
            # print(cnt, feature, target)
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            target = target.squeeze()  # 원핫으로 만들면 필요없음

            # 학습
            pre_target = model(feature)
            # print(f'pre_target => {pre_target.shape}, {pre_target.ndim}D')
            # print(f'target => {target.shape}, {target.ndim}D')
        
            # 손실함수
            loss = LOSS_FN(pre_target, target)
            val_loss.append(loss)

            # 배치 단위 학습 진행 메세지 출력
            # print(f'[Valid {cnt} Batch Loss] ==> {loss}')

    # 에포크 단위 학습 진행 메세지 출력
    acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes = 3)
    print(f'[Valid Loss] ==> {loss}, [Valid Accuracy] ==> {acc}')

    return val_loss

In [72]:
# 예측 함수
def predict():
    pass

In [73]:
# 지정된 횟수만큼 처음부터 끝까지 학습 및 검증 진행
# 목표 : 최적(Error 최소화)의 W, b를 가진 모델 완성
for eps in range(EPOCHS):
    # 학습
    train_loss = training()
    
    # 검증
    valid_loss = testing(validDL)
    
    print(f'[{eps}/{EPOCHS}] TRAIN : {sum(train_loss)/len(train_loss)},\
          VALID : {sum(valid_loss)/len(valid_loss)}')
    
    mean_valid_loss = sum(valid_loss)/len(valid_loss)
    SCHEDULER.step(mean_valid_loss)
    # 조기종료 기능 => 조건 : val_loss가 지정된 횟수(예 : 5)이사아 개선이 안되면 학습 종료
    if SCHEDULER.num_bad_epochs > SCHEDULER.patience:
        print(f'Early stopping at epoch {eps}')
        break

[Train Loss] ==> 0.9306038022041321 
[Valid Loss] ==> 0.8951473236083984 [Valid Accuracy] 0.800000011920929
[0/50] TRAIN : 0.9957864880561829,          VALID : 0.9003342390060425
[Train Loss] ==> 0.7772628664970398 
[Valid Loss] ==> 0.7689316272735596 [Valid Accuracy] 0.800000011920929
[1/50] TRAIN : 0.8667465448379517,          VALID : 0.7775547504425049
[Train Loss] ==> 0.6411141157150269 
[Valid Loss] ==> 0.6595633625984192 [Valid Accuracy] 0.800000011920929
[2/50] TRAIN : 0.7510023713111877,          VALID : 0.6682748794555664
[Train Loss] ==> 0.5263972282409668 
[Valid Loss] ==> 0.57602459192276 [Valid Accuracy] 0.800000011920929
[3/50] TRAIN : 0.6418629884719849,          VALID : 0.5774166584014893
[Train Loss] ==> 0.4374598562717438 
[Valid Loss] ==> 0.526907205581665 [Valid Accuracy] 0.800000011920929
[4/50] TRAIN : 0.5473095178604126,          VALID : 0.5162461400032043
[Train Loss] ==> 0.3806770443916321 
[Valid Loss] ==> 0.4898742139339447 [Valid Accuracy] 0.800000011920929


[7] 테스트 진행